In [1]:
# connecting database
import sqlite3
db = sqlite3.connect("APAD_proj.db")

In [2]:
# check if the user already exists in the database based on the email id
def email_check(db,email):
    cursor = db.cursor()
    while(1):
        findEmail = '''SELECT * FROM user where email_id = ?'''
        cursor.execute(findEmail, (email,))
        if cursor.fetchall():
            print("Email already registered, enter again")
            email = str(input("Please provide your email-address: "))
        else:
            return email

In [3]:
# check if the entered email is admin's email or not. This function is called where only admin is allowed to make changes
def admin_check(db,email):
    cursor = db.cursor()
    findEmail = '''SELECT email_id FROM user where first_name = ?'''
    cursor.execute(findEmail, ("admin",))
    admin = cursor.fetchone()[0]
#     print(admin)
    while email != admin:
        print("Only admin is authorized to add a venue")
        email = str(input("Enter the admin's email address: "))
    
    return email

In [4]:
# verify new user's password in order to create account
def password_verify(pwd):
    while pwd[0]!=pwd[1]:
        print("Your passwords didn't match, enter again")
        pwd[0] = str(input("Enter the password: ")) # NEED to add constraints
        pwd[1] = str(input("Enter the password again to recheck: "))
    return pwd[0], pwd[1]
        

In [5]:
# create a new user
def new_user(db, inputs):

    cursor = db.cursor()
    query = '''INSERT INTO user(email_id,first_name,last_name,contact_number, zip_code, password)
                    VALUES(?,?,?,?,?,?)'''

    cursor.execute(query, (inputs[:6]))  
    db.commit()


In [6]:
# taking inputs from user to create a new account and make a list of all the entries. Output from this function is fed to new_user(db, inputs)
def user_input_form():
    email = str(input("Please provide your email-address: "))
    # check if the email already exists
    email = email_check(db, email)
    first_name = str(input("Please Enter your first name: "))
    last_name = str(input("Please Enter your last name: "))
    contact_number = str(input("Please provide a valid contact number: "))
    zip_code = str(input("Your zip code: "))
    password = str(input("Enter the password: ")) # NEED to add constraints
    password_check = str(input("Enter the password again to recheck: "))
    pwd = [password, password_check]
    # verify password function call
    password, password_check = password_verify(pwd)
    inputs = [email, first_name, last_name, contact_number, zip_code, password, password_check]
    
    return inputs

In [ ]:
# Single User input
inputs = user_input_form()
new_user(db, inputs)

In [7]:
# insert a new venue in venue table
def new_venue(db, inputs):
    
    cursor = db.cursor()
    
    query = '''INSERT INTO venue(venue_name, address, zip_code, contact_number, description, open_time, close_time, games_total_count, games_available_count)
                VALUES(?,?,?,?,?)'''

    # cursor = db.cursor()
    cursor.execute(query, inputs_venue[1:]) 
    db.commit()


In [8]:
# checking if the person trying to create a venue is admin or not. If admin, then input all the values required to create a new venue
def add_venue_form():
    email = str(input("Please enter your email address: "))
    email1 = admin_check(db,email)
    venue_name = str(input("Please Enter the venue name: "))
    address = str(input("Please Enter the venue address: "))
    zip_code = str(input("Venue zip code: "))
    contact_number = str(input("Please provide a contact number: "))
    description = str(input("Enter the venue description (not more than 200 characters): ")) # Need to limit the length of input
    open_time = str(input("Please Enter the start time in hrs: "))
    close_time   = str(input("Please Enter the end time in hrs: "))
    games_total_count = int(input("Please enter the total number of games that can be played in single time slot: "))
    games_available_count = int(input("Please enter the available number of games that can be played in single time slot: "))
    
    inputs_venue = [email1,venue_name,address,zip_code,contact_number,description,open_time,close_time,games_total_count, games_available_count]
    return inputs_venue

In [9]:
inputs_venue = add_venue_form()
new_venue(db, inputs_venue)

Please enter your email address: pp@gmail.com
Only admin is authorized to add a venue
Enter the admin's email address: admin@gmail.com
Please Enter the venue name: Zilker Park
Please Enter the venue address: 2100 Barton Springs Rd, Austin, TX 78704
Venue zip code: 78704
Please provide a contact number: 7377011298
Enter the venue description (not more than 200 characters): Very big park
Please Enter the start time in hrs: 7
Please Enter the end time in hrs: 20
Please enter the total number of games that can be played in single time slot: 3
Please enter the available number of games that can be played in single time slot: 3


OperationalError: table venue has no column named open_time

In [ ]:
# verify if the user is registered already or not if he tries to create an event
def email_verify(db, email):
    cursor = db.cursor()
    
    findEmail = '''SELECT email_id FROM user where email_id = ?'''
    cursor.execute(findEmail, (email,))
    
    if cursor.fetchall():
        return email
    else:
        print("You can't create an event yet. Create an account first")
        inputs = user_input_form()
        new_user(db, inputs)
        return inputs[0]

In [ ]:
# show the event categories to choose from - till now we have 3 categories: Play/Watch/Workshop
def fetch_event_cat(cursor):
    event_cat_query = '''SELECT event_category_name FROM event_category'''
    cursor.execute(event_cat_query)
    event_cat = cursor.fetchall()
    print("Please select a category from the following: ") 
    for i in range(len(event_cat)):
        print(str(i) +":" + event_cat[i])  

In [ ]:
# show the all the venues to choose from 
def fetch_venue(cursor):
    venue_query = '''SELECT venue_name FROM venue'''
    cursor.execute(venue_query)
    venues = cursor.fetchall()
    print("Please select a venue from the following: ") 
    for i in range(len(venues)):
        print(str(i) +":" + venues[i])
    return venues

In [ ]:
# show all the sports available in the table
def fetch_sport(cursor):
    sport_query = '''SELECT sport_name FROM sports_cat'''
    cursor.execute(sport_query)
    sports = cursor.fetchall()
    print("Please select a sport from the following: ") 
    for i in range(len(sports)):
        print(str(i) +":" + sports[i])
    return sports

In [ ]:
# creates a new event from the list output of event_create_inputs(db) function
def new_event(db,inputs):
    
    cursor = db.cursor()

    if inputs[9] == "create":

        host_insert_flag = '''INSERT INTO events(event_cat_id,venue_id,event_name, date, start_time,end_time,
                                              user_id, host_flag, member_flag, sports_cat_id, event_desc, capacity_avail)
                       VALUES(?,?,?,?,?,?,?,?,?,?,?,?)'''


        cursor.execute(host_insert_flag, (inputs[7],inputs[8],inputs[1],inputs[2],inputs[3],inputs[4],inputs[0],1,0,inputs[10],inputs[5], inputs[6]))  

        db.commit()
        
        # -1 the count of games_available count in venue table
        
        update_game_avail_ct  = '''UPDATE TABLE venue set games_available_count = games_available_count -1
                                    where venue_id = ?'''
        cursor.execute(host_insert_flag,inputs[8])
        db.commit()


    elif inputs[9] == "join":

        find_event = '''SELECT * FROM events where date = ?
                            and start_time = ? and end_time = ?'''
        cursor.execute(find_event, (inputs[2], inputs[3], inputs[4],))
        p = cursor.fetchall()
    else:
        print("enter a valid option")

In [ ]:
# inputs to create a new event
def event_create_inputs(db):
    
    count = 0
    cursor = db.cursor()
    db.row_factory = lambda cursor, row: row[0]
    
    #Input the email of the user
    email = str(input("Please provide your email-address: "))

    #Check if it's an existing user
    email = email_verify(db,email)
    
    #Get the user_id from the database
    user_id_query = '''SELECT user_id FROM user where email_id = ?'''
    cursor.execute(user_id_query, (email,))
    user_id = cursor.fetchone()
    
    event_name = str(input("Please Enter the event name: "))
    
    #Fetch the list of sports 
    sports = fetch_sport(cursor)
    
    #Input for sport. Only accept if it exists!
    while(1):
        sport_name = str(input("Please Enter the sport name: "))
        for i in sports:
            if (sport_name == i):
                count=1
                break
        if count==1:
            break
        else:
            print("Please enter the correct sport from choices above!")

    #Get the sport id from the database
    sports_cat_id_query = '''SELECT sports_cat_id FROM sports_cat where sport_name = ?'''
    cursor.execute(sports_cat_id_query, (sport_name,))
    sports_cat_id = cursor.fetchone()
    
    event_date = str(input("Please Enter the event date: "))
    start_time = str(input("Please Enter the start time in hrs: "))
    end_time   = str(input("Please Enter the end time in hrs: "))
    event_desc = str(input("Please Enter the event description: "))
    capacity   = str(input("Please Enter the capacity avail: "))
    
    #Fetch all event categories
    fetch_event_cat(cursor)
    
    #Input for event category. Only accept if it exists!
    while(1):
        event_category_name = str(input("Please Enter the event category name : "))
        if (event_category_name == "watch" or event_category_name == "play" or event_category_name == "workshop" ):
            break
        else:
            print("Please enter the right category from choices above!")
    
    #Get the Event cat id from the database
    event_cat_id_query = '''SELECT event_cat_id FROM event_category where event_category_name = ?'''
    cursor.execute(event_cat_id_query, (event_category_name,))
    event_cat_id = cursor.fetchone()
    
    #Fetch all the venues
    venues = fetch_venue(cursor)
    
    #Input for venue. Only accept if it exists!
    count=0
    while(1):
        venue_name = str(input("Please Enter the venue name: "))
        for i in venues:
            if (venue_name == i):
                count=1
                break
        if count==1:
            break
        else:
            print("Please enter the correct venue from choices above!")
    
    #Get the venue id from the database
    venue_id_query = '''SELECT venue_id FROM venue where venue_name = ?'''
    cursor.execute(venue_id_query, (venue_name,))
    venue_id = cursor.fetchone()
    
    create_or_join = str(input("Do you want to create/join the game: ")).lower()
    
    inputs = [user_id,event_name,event_date,start_time,end_time,event_desc,capacity,event_cat_id,venue_id,create_or_join,sports_cat_id]
    
    return inputs

In [ ]:
inputs2 = event_create_inputs(db)
if inputs2:
    new_event(db,inputs2)

In [ ]:
from datetime import datetime, timedelta

In [ ]:
booked_slots = [(datetime(2019,7,26,10), datetime(2019,7,26,12)),
                (datetime(2019,7,26,14,30),datetime(2019,7,26,17))]

total_hours = (datetime(2019, 7, 26, 9), datetime(2019, 7, 26, 20))
duration=timedelta(hours=1)
slots = sorted([(total_hours[0],total_hours[0])] + booked_slots + [(total_hours[1],total_hours[1])])
print(slots)
for start, end in ((slots[i][1], slots[i+1][0]) for i in range(len(slots)-1)):
    assert start <= end, "Cannot attend all appointments"
    while start + duration <= end:
        print ("{:%H:%M} - {:%H:%M}".format(start, start + duration))
        start += duration